<a href="https://www.kaggle.com/code/sofiamatias/learning-equality-model-sm?scriptVersionId=117773714" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Kaggle Challenge - Learning Equality

https://www.kaggle.com/competitions/learning-equality-curriculum-recommendations/overview

## Goal of the Competition

The goal of this competition is to streamline the process of matching educational content to specific topics in a curriculum. You will develop an accurate and efficient model trained on a library of K-12 educational materials that have been organized into a variety of topic taxonomies. These materials are in diverse languages, and cover a wide range of topics, particularly in STEM (Science, Technology, Engineering, and Mathematics).

Your work will enable students and educators to more readily access relevant educational content to support and supplement learning.

## Submission File

For each **topic_id** in the test set, you must predict a space-delimited list of recommended **content_ids** for that topic. The file should contain a header and have the following format:

~~~
topic_id,content_ids
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c_76231f9d0b5e
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c_ebb7fdf10a7e
t_00069b63a70a,c_11a1dc0bfb99
...
~~~

## Scoring
Mean F2 score

## Evaluation Metric - Efficiency Scoring
We compute a submission's efficiency score by:

\begin{equation} \text{Efficiency} = \frac{1}{ \text{Benchmark} - \max\text{F2} }\text{F2} + \frac{1}{32400}\text{RuntimeSeconds} \end{equation}


where **F2** is the submission's score on the main competition metric, **Benchmark** is the score of the benchmark sample_submission.csv, **maxF2** is the maximum  of all submissions on the Private Leaderboard, and **RuntimeSeconds** is the number of seconds it takes for the submission to be evaluated. The objective is to minimize the efficiency score.

During the training period of the competition, you may see a leaderboard for the public test data in the following notebook, updated daily: Efficiency Leaderboard. After the competition ends, we will update this leaderboard with efficiency scores on the private data. During the training period, this leaderboard will show only the rank of each team, but not the complete score.

# How To Solve This Challenge

## Model Train

* Calculate embeddings: for topics (title, description) and contents (title, description and text), use SentenceTransformer. Split sentences by language.
* Use KNN model: train model with content embeddings and use topic embeddings to predict content matches, use k=10, 20, 30, 50
* Calculate F2 to choose best k.
* Submit predictions
* Set X_train, X_test, y_train, y_test: split by category (use 'aligned' for validation), set has_content = True for validation. Check dimensions
* Use all features for X, including topic_id and content_id, topic_title and content_title. Use the KNN predictions to "mount" X and y. Check dimensions.
* Use correlations to get y: if KNN topic-content match with correlations, y is 1, else is 0
* Use SVM RBF or KNN again for multiclass classification
* Use RandomizedSearch to get best hyperparameters. Use model score "recall". Split the dataset in languages for training: train model for each language individually
* Calculate F2 and see if we've got improvements from KNN.
* Submit predictions

## Submissions

* Use submission sample and get topics and contents
* Apply KNN model to get topic-content matches. Get contents per topic
* Filter "good matches" with second classification model
* Compare predicts with submission sample and calculate F2

## Efficiency

* Use KNN and 2nd model without GPU and check if it takes a long time to calculate sample submissions

# Data

## Imports

In [3]:
import sys, os
sys.path.append("../input/sentence-transformer-package/sentence-transformers-2.2.2/sentence-transformers-2.2.2") 
import sentence_transformers

In [4]:
!pip install -q /kaggle/input/loguru-lib-ds/loguru-0.5.3-py3-none-any.whl

In [5]:
import pandas as pd
import numpy as np
import string
import torch
from pathlib import Path
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import precision_score, recall_score, fbeta_score
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate, train_test_split

In [15]:
DATA_PATH = Path.cwd().parent.resolve()
drive_path = DATA_PATH / 'input'
files_path = DATA_PATH / 'input/learning-equality-curriculum-recommendations'
dataset_path = drive_path / 'learningequalityfiles'
work_path = DATA_PATH / 'working'
model_path = '/kaggle/input/sentence-transformer-package/ST-all-MiniLM-L6-v2-trained/ST-all-MiniLM-L6-v2-trained'

# Data Collection

## Load dataframes

In [20]:
def read_file_in_chunks(filename):
        
    list_chunks = []
    chunksize = 10000
    with pd.read_csv(filename, chunksize=chunksize, index_col=0) as reader:
        for chunk in reader:
            list_chunks.append(chunk)
    df = pd.concat(list_chunks)
    return df

In [23]:
# loading dataframes from existing .csv files

print (f"\nLoading dataframes...")

low=0 #row number to start dataset subset
high=1000 #row number to end dataset subset

df_content = pd.DataFrame([])
df_topics = pd.DataFrame([])
df_corr = pd.DataFrame([])
df_sample = pd.DataFrame([])
for path, subdirs, files in os.walk(drive_path):
    for file in files:
        if file.endswith(".csv") == True:
            filepath = os.path.join(path, file)
            df = read_file_in_chunks(filepath)
            if set(['copyright_holder', 'kind']).issubset(df.columns):
                df_content = df[low:high].fillna({"title": "", "description": ""})
                display(df_content)
                print (f"\nLoaded 'df_content' from {filepath}")
            elif set(['channel', 'parent']).issubset(df.columns):
                df_topics = df[low:high].fillna({"title": "", "description": ""})
                display(df_topics)
                print (f"\nLoaded 'df_topics' from {filepath}")
            elif ('content_ids' in df.columns) and ('sample' not in file):
                df_corr = df
                display(df_corr)
                print (f"\nLoaded 'df_corr' from {filepath}")
            if ('sample' in file):
                df_sample = df
                display(df_sample)
                print (f"\nLoaded 'df_sample' from {filepath}")


Loading dataframes...


,content_ids
topic_id,
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
t_00069b63a70a,c_11a1dc0bfb99
t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
t_4054df11a74e,c_3695c5dc1df6 c_f2d184a98231



Loaded 'df_sample' from /kaggle/input/learning-equality-curriculum-recommendations/sample_submission.csv


,title,description,channel,category,level,language,parent,has_content
id,,,,,,,,
t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
t_000095e03056,Unit 3.3 Enlargements and Similarities,,b3f329,aligned,2,en,t_aa32fb6252dc,False
t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
t_00069b63a70a,Transcripts,,6e3ba4,source,3,en,t_4054df11a74e,True
t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True
...,...,...,...,...,...,...,...,...
t_0353f7c74647,Peninsular Plateau of India,,ef2088,supplemental,5,en,t_cc5779490262,True
t_0355125ac080,Techniques of Control,,ef2088,supplemental,5,en,t_96a9a6b73988,True
t_035617cf7ce2,2.2 - Iterative patterns & processes,,c6dab5,source,3,en,t_41e08e0fb197,False



Loaded 'df_topics' from /kaggle/input/learning-equality-curriculum-recommendations/topics.csv


...


Loaded 'df_corr' from /kaggle/input/learning-equality-curriculum-recommendations/correlations.csv


,title,description,kind,text,language,copyright_holder,license
id,,,,,,,
c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,NaN,es,NaN,NaN
c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,NaN,it,NaN,NaN
c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,NaN,es,NaN,NaN
c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA
...,...,...,...,...,...,...,...
c_01b62d0b9b40,Cómo pegar un ensamble de caja y espiga o de c...,Con este vídeo aprenderás a pegar piezas de ma...,document,\nCómo pegar un ensamble de caja\ny espiga o ...,es,Sikana Education,CC BY-NC-ND
c_01b6e399bea7,Lección 29: La energía lumínica y energía química,,exercise,La fotosíntesis provocó el cambio en la atmósf...,es,Publicado por el Lic. Edelberto Andino(edelber...,CC BY-NC-SA
c_01b71b0ee2ba,Transformation et parallélogramme - Triangles ...,,html5,NaN,fr,NaN,NaN



Loaded 'df_content' from /kaggle/input/learning-equality-curriculum-recommendations/content.csv


,content_ids
topic_id,
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
t_00069b63a70a,c_11a1dc0bfb99
t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
...,...
t_fff830472691,c_61fb63326e5d c_8f224e321c87
t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...
t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5



Loaded 'df_corr' from /kaggle/input/learningequalityfiles/correlations.csv


In [24]:
#Sampling 'df_topics' and 'df_content' with sample_submission
if ~df_sample.empty:
    df_topics = df_topics.merge(df_sample, how='inner', left_index=True, right_index=True)
    print (f"\nFiltered 'df_topics' according to 'sample_submission'")
if df_content.empty and df_topics.empty:
    print ('Error: there are no topics or content files to work with')
    quit()


Filtered 'df_topics' according to 'sample_submission'


,title,description,channel,category,level,language,parent,has_content,content_ids
t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
t_00069b63a70a,Transcripts,,6e3ba4,source,3,en,t_4054df11a74e,True,c_11a1dc0bfb99
t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...


In [25]:
# load 'correlations' data into pandas dataframe
# Restructure 'df_corr' (correlations): explode the target column to be more practical to use - one topic -> one content
y = df_corr.copy()
y['content_ids'] = y.content_ids.str.split(' ')
y = y.explode('content_ids')
y.reset_index(inplace=True)
print (f"\nChanged 'df_corr' to exploded 'y'")
display(y)


Changed 'df_corr' to exploded 'y'


,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d
1,t_00004da3a1b2,c_376c5a8eb028
2,t_00004da3a1b2,c_5bc0e1e2cba0
3,t_00004da3a1b2,c_76231f9d0b5e
4,t_00068291e9a4,c_639ea2ef9c95
...,...,...
279914,t_fff9e5407d13,c_d64037a72376
279915,t_fffbe1d5d43c,c_46f852a49c08
279916,t_fffbe1d5d43c,c_6659207b25d5
279917,t_fffe14f1be1e,c_cece166bad6a


## Create Topic Breadcrumbs Functions

In [26]:
# define some helper functions and classes to aid with data traversal

def get_topic_breadcrumbs (df_topic):
    print (f"Creating breadcrumbs...")
    topic_breadcrumbs=[]
    topic_ids=[]
    if 'topic_id' == df_topics.index.name:
        col_name = 'topic_id'
    else:
        col_name = 'id'
    for i, topic in enumerate(df_topic.index):
        if (i % 5000 == 0) and (i > 0):
            print (f'Created {i} breadcrumbs...')
        if df_topic.loc[topic].parent in df_topic.index:
            topic_ids.append (topic)
            topic_breadcrumbs.append (Topic(topic).get_breadcrumbs(separator=' '))
    print (f"Creating dataframe from breadcrumbs 'df_topics_breadcrumbs'...")
    df_topics_breadcrumbs = pd.DataFrame (topic_breadcrumbs, 
                                          index = topic_ids,
                                          columns = ['topic_breadcrumbs'])
    return df_topics_breadcrumbs


def print_markdown(md):
    display(Markdown(md))

class Topic:
    def __init__(self, topic_id):
        self.id = topic_id

    @property
    def parent(self):
        parent_id = df_topics.loc[self.id].parent
        if pd.isna(parent_id):
            return None
        else:
            return Topic(parent_id)

    @property
    def ancestors(self):
        ancestors = []
        parent = self.parent
        while (parent is not None) and (parent.id in df_topics.index):
            ancestors.append(parent)
            parent = parent.parent
        return ancestors

    @property
    def siblings(self):
        if not self.parent:
            return []
        else:
            return [topic for topic in self.parent.children if topic != self]

    @property
    def content(self):
        if self.id in df_corr.index:
            return [ContentItem(content_id) for content_id in df_corr.loc[self.id].content_ids.split()]
        else:
            return tuple([]) if self.has_content else []

    def get_breadcrumbs(self, separator=" >> ", include_self=True, include_root=True):
        ancestors = self.ancestors
        if include_self:
            ancestors = [self] + ancestors
        if not include_root:
            ancestors = ancestors[:-1]
        return separator.join(reversed([a.title for a in ancestors]))

    @property
    def children(self):
        return [Topic(child_id) for child_id in df_topics[df_topics.parent == self.id].index]

    def subtree_markdown(self, depth=0):
        markdown = "  " * depth + "- " + self.title + "\n"
        for child in self.children:
            markdown += child.subtree_markdown(depth=depth + 1)
        for content in self.content:
            markdown += ("  " * (depth + 1) + "- " + "[" + content.kind.title() + "] " + content.title) + "\n"
        return markdown

    def __eq__(self, other):
        if not isinstance(other, Topic):
            return False
        return self.id == other.id

    def __getattr__(self, name):
        return df_topics.loc[self.id][name]

    def __str__(self):
        return self.title
    
    def __repr__(self):
        return f"<Topic(id={self.id}, title=\"{self.title}\")>"


class ContentItem:
    def __init__(self, content_id):
        self.id = content_id

    @property
    def topics(self):
        return [Topic(topic_id) for topic_id in df_topics.loc[df_corr[df_corr.content_ids.str.contains(self.id)].index].index]

    def __getattr__(self, name):
        return content_df.loc[self.id][name]

    def __str__(self):
        return self.title
    
    def __repr__(self):
        return f"<ContentItem(id={self.id}, title=\"{self.title}\")>"

    def __eq__(self, other):
        if not isinstance(other, ContentItem):
            return False
        return self.id == other.id

    def get_all_breadcrumbs(self, separator=" >> ", include_root=True):
        breadcrumbs = []
        for topic in self.topics:
            new_breadcrumb = topic.get_breadcrumbs(separator=separator, include_root=include_root)
            if new_breadcrumb:
                new_breadcrumb = new_breadcrumb + separator + self.title
            else:
                new_breadcrumb = self.title
            breadcrumbs.append(new_breadcrumb)
        return breadcrumbs

## Data Cleaning

## Data cleaning params

In [27]:
levels = {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven', 8: 'eight',
          9: 'nine', 10: 'ten', 0: 'zero'}
topics_cols = ['title', 'description', 'topic_breadcrumbs']
content_cols = ['title', 'description', 'text']
cat_for_val = 'aligned'
print (f"\nLoaded cleaning parameters")


Loaded cleaning parameters


## Data cleaning functions

In [28]:
# Clean up text
def clean_text(text_col):
    """
    Clean ponctuation and special chars from a dataframe column
    """
    punctuations = string.punctuation
    text_col = text_col.str.replace('\W', ' ', regex=True)
    for punct in string.punctuation:
        text_col = text_col.str.replace(punct, ' ', regex=True)
    return text_col

### Topics

*   Changing 'level' values from integers to strings
*   Clean strings: ponctuation and special chars (\n\t...)
*   Sort values by language

In [29]:
def build_topic_features (topics_df, df_topics_breadcrumbs, levels, topic_cols):
    """
    Create 'topics_features' from df_topics, clean parameters and functions
    """
    print (f"\nCreating and cleaning topic features...")
    topics_features = topics_df.copy()
    topics_features = topics_features.replace ({'level': levels})
    topics_features = topics_features.merge (df_topic_breadcrumbs, how='outer', right_index=True, left_index=True)
    for col in topics_cols:
        topics_features[col] = clean_text(topics_features[col])
    topics_features['sentences'] = topics_features[topics_cols].apply(lambda x: '.'.join(x.dropna().astype(str)), axis=1)
    topics_features = topics_features.drop(columns=['parent'] + topics_cols) 
    print (f"\nCreated 'topic_features'")
    display (topics_features.head())
    
    return topics_features


### Contents


*   Remove columns 'copyright_holder' and 'language' (for 'language' assume that topic-content match with correlations is same language)
*   Clean strings: ponctuation and special chars (\n\t...)

In [30]:
def build_content_features (content_df, content_cols):
    """
    Create 'content_features' from df_content, clean parameters and functions
    """
    print (f"\nCreating and cleaning content features...")
    content_features = content_df.copy()
    for col in content_cols:
        content_features[col] = clean_text(content_features[col])
    content_features['sentences'] =  content_features[content_cols].apply(lambda x: '.'.join(x.dropna().astype(str)), axis=1)
    content_features = content_features.drop(columns=['copyright_holder', 'license'] + content_cols)
    print (f"\nCreated 'content_features'")
    display (content_features.head())
    
    return content_features

# Model Train

### Models params

In [31]:
#General
device = 'cpu'
if torch.cuda.is_available():
    device = torch.device('cuda:0')
#Sentence Transformer
all_mini = 'all-MiniLM-L6-v2'
paraphrase = 'paraphrase-multilingual-MiniLM-L12-v2'
trained_model = model_path
# KNN
k = [5,10,20,30,50,100]
alg = 'kd_tree' #'ball_tree','brute'
leaf = 10
print (f"\nLoaded models parameters")


Loaded models parameters


### Scoring functions

In [32]:
def model_scoring (y_test, y_pred, average='macro'):
    """
    Calculate precision, recall and f2-score for y_test and y_pred
    """
    precision = precision_score(y_test['content_ids'], y_pred['content_ids'], average=average)
    print ('Precision:', precision)
    recall = recall_score(y_test['content_ids'], y_pred['content_ids'], average=average)
    print ('Recall:', recall)
    F2macro = fbeta_score(y_test['content_ids'], y_pred['content_ids'], beta=2, average='macro')
    print ('F2 macro:', F2macro)
    F2micro = fbeta_score(y_test['content_ids'], y_pred['content_ids'], beta=2, average='micro')
    print ('F2 micro:', F2micro)

In [33]:
def calculate_Fscore(prediction_df, actual_df):
    
    prediction_df['content_ids'] = prediction_df.content_ids.str.split(' ')
    prediction_df.columns=['topic_id', 'content_ids_pred']
    actual_df['content_ids'] = actual_df.content_ids.str.split(' ')
    actual_df.columns=['topic_id', 'content_ids_actual']
    df = pd.merge(prediction_df, actual_df, how='inner', on='topic_id')
    df['correct_pred'] = df[['content_ids_pred', 'content_ids_actual']].apply(lambda x: len([d for d in x[0] if d in x[1]]), axis=1)
    df['precision'] = df['correct_pred']/(df.content_ids_actual.str.len() + 1e-7)
    df['recall'] = df['correct_pred']/(df.content_ids_pred.str.len() + 1e-7)
    for beta in [0.5, 1, 2]:
        df['f'+str(beta)] = ((1 + beta**2) * df['precision'] * df['recall'])/((beta**2 * df['precision']) + df['recall'] + 1e-7) 
    
    return df

In [34]:
#example_pred = [{ 'topic_id': 't1', 'content_ids': 'c1 c2 c3 c10 c12 c13 c14'}, { 'topic_id': 't2', 'content_ids': 'c1 c2 c3 c14'}]
#example_actual = [{ 'topic_id': 't1', 'content_ids': 'c1 c2 c3 c4 c5'}, { 'topic_id': 't2', 'content_ids': 'c2 c4 c5'}]
#calculate_Fscore(pd.DataFrame(example_pred), pd.DataFrame(example_actual))

## Calculate Embeddings Functions

* Use output as pytorch tensor and normalize tensor embeddings
* Convert to numpy array

In [35]:
def get_embeddings(topics_features, content_features):
    """
    Calculate embeddings for both topics and content
    From text in columns:
    topic_cols=['title','description', 'topic_breadcrumbs'] 
    content_cols=['title','description', 'text']
    """
    device = 'cpu'
    if torch.cuda.is_available():
        device = torch.device('cuda:0')

    encoder = SentenceTransformer(trained_model)
    topics_id_for_embeddings=[]
    topics_embeddings=[]
    content_ids_for_embeddings=[]
    content_embeddings=[]
    topics_aux = []
    content_aux = []

    topics_sentences = topics_features['sentences']
    print (f'\nGetting embeddings for {len(topics_sentences)} sentences from topics')
    # topics embeddings
    topics_aux = encoder.encode(topics_sentences, convert_to_tensor=True, show_progress_bar=False)
    topics_id_for_embeddings = topics_sentences.index
    topics_embeddings = util.normalize_embeddings(topics_aux.to(device)).detach().cpu().numpy()
    # content embeddings
    content_sentences = content_features['sentences']
    print (f'\nGetting embeddings for {len(content_sentences)} sentences from contents')
    content_ids_for_embeddings = content_sentences.index
    content_aux = encoder.encode(content_sentences, convert_to_tensor=True, show_progress_bar=True)
    content_embeddings = util.normalize_embeddings(content_aux.to(device)).detach().cpu().numpy()
    
    return topics_embeddings, content_embeddings, topics_id_for_embeddings, content_ids_for_embeddings

## Apply Nearest Neighbours Functions

* Use unsupervised NN. Use GPU when available, otherwise use sklearn NN

### topic-content predictions functions (df's)

In [36]:
def get_preds(list_aux, topics_id_for_embeddings):
    """
    df_preds = Dataframe with topic_id's and KNN chosen content_ids
    """
    df = pd.DataFrame(zip (list_aux, topics_id_for_embeddings), columns=['content_ids', 'topic_id'])
    df = df.iloc[:,[1,0]]
    df.content_ids = df.content_ids.str.join(' ')
    df = df.fillna('')
    return df

In [37]:
def get_true(df_corr, df):
    """
    Ground truth
    df_true = Dataframe with same topic_id's from 'get_preds' function but content_id's taken from correlations, 
    for score calculations. Returns the format defined in 'correlations.csv' but as dataframe.
    """
    df_true = df_corr.copy().reset_index().merge (df, how='right', on='topic_id')[['topic_id', 'content_ids_x']].rename(columns={'content_ids_x': 'content_ids'})
    df_true = df_true.fillna('')
    return df_true

### Calculate k neighbors / F2 scoring function

In [57]:
def k_neighbors (k, topics_embeddings, content_embeddings, topics_id_for_embeddings, content_ids_for_embeddings, df_corr):
    """
    Calculate k neighbors of contents_id's for each topic_id
    """
        
    device = 'cpu'
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    if device == 'cpu':
        from sklearn.neighbors import NearestNeighbors
    else:
        import cudf
        import cuml
        from cuml.neighbors import NearestNeighbors

    # Check if number of samples is below preset K value and reconfigure content_embeddings 
    topics_count = int(sum(x.size for sublist in topics_embeddings for x in sublist)/384)
    #Model
    print ('\n K value:', k)
    model = NearestNeighbors(n_neighbors=k,
                             n_jobs=-1)

    print ('\nFitting KNN model...')
    # Fit and Predictions
    print (f"\nGetting content_ids for {topics_count} topics... ")
    nbrs = model.fit(content_embeddings)
    _, indices = nbrs.kneighbors(topics_embeddings)
    content_id_list = np.array([content_ids_for_embeddings[val] for row in indices for val in row]).reshape(indices.shape)
    
    print ('\nCalculating scores...')
    #Define dataframes for predictions and ground truth
    df_preds = get_preds(content_id_list, topics_id_for_embeddings)
    df_true = get_true(df_corr, df_preds)
    fscore = calculate_Fscore(df_preds.copy(), df_true.copy())
    print ('\nTopics to match content:', topics_count+1)
    print ('\nCorrect predictions:', fscore.correct_pred.sum())
    print ('\nF2 Score:', fscore.f2.mean())
    #print ('\n\nF2 score by sklearn functions:')
    #model_scoring (df_preds.copy(), df_true.copy(), average='micro')
    print ('\n')
    
    return model, df_preds, df_true, fscore

## K parameter vs F2 score

# Setup Features and Target Functions

* Set X_train, X_test, y_train, y_test: split by category (use 'aligned' for validation); set has_content = True for validation. Check dimensions
* Use all features for X, including topic_id and content_id, topic_title and content_title
* Use the KNN predictions to "mount" X and y. Check dimensions.
* Use correlations to get y: if KNN topic-content match with correlations, y is 1, else is 0
* Use LGBMClassifier for multiclass classification

In [39]:
def get_target(preds):
    """
    Get a binary target from comparing NN predictions y_preds with ground truth 'y' from 'df_corr'
    """
    preds_aux = preds.copy()
    preds_aux.content_ids = preds_aux.content_ids.str.split(' ')
    preds_aux = preds_aux.explode('content_ids')
    y_int = preds_aux.merge (y, on='topic_id', how='left')
    y_int['match'] = (y_int.iloc[:,1] == y_int.iloc[:,2]).astype (int) 
    y_int = y_int.groupby(['topic_id', 'content_ids_x']).sum().reset_index()
    return y_int

In [124]:
def get_features (y_aux, topics_features, content_features):
    """ 
    Get features for classification. Using topic_id, topic_title, content_id and content_title, 
    among remaining class features. Returns Xcolumns=X. and y_target.
    """
    print ("\nSorting features for X...\n")
    X = y_aux.copy()
    X = X.merge (topics_features.reset_index(), how='left', left_on='topic_id', right_on='index')
    X['topic_title'] = X.sentences.str.split('.').str[0]
    X.drop(columns=X.iloc[:,[3,9,10]].columns.tolist(), inplace=True)
    content_aux = contents_features.reset_index().drop(columns=contents_features.iloc[:,1].name) 
    X = X.merge (content_aux, how='left', left_on='content_ids_x', right_on=contents_features.reset_index().iloc[:,0].name)
    X = X[X.has_content == True]
    X['content_title'] = X.sentences.str.split('.').str[0]
    X.drop(columns=X.iloc[:,[9,12]].columns.tolist(), inplace=True)
    X.columns = X.columns.str.replace('_x', '')
    X = X.sort_values('language')
    print ("\nSorting binary y_target...\n\n")
    y_target = X.iloc[:,2]
    #X=X.drop(columns=X.iloc[:,2].columns.tolist(), inplace=True)
    print ("\nX and y_target are defined.")
    return X, y_target

# Feature Preprocessing



In [85]:
#Getting parameters
K = 100 #k[4] #50

In [59]:
#Getting topics breadcrumbs or topics context
print ("\n* Getting 'topics_breadcrumbs' dataframe...\n")
df_topic_breadcrumbs = get_topic_breadcrumbs (df_topics)

#Cleaning data and getting topics and contents features
print ('\n* Calculating topics_features dataframe...\n')
topics_features = build_topic_features (df_topics, df_topic_breadcrumbs, levels, topics_cols)

print ('\n* Calculating content_features dataframe...\n')
contents_features = build_content_features (df_content, content_cols)


* Getting 'topics_breadcrumbs' dataframe...

Creating breadcrumbs...
Creating dataframe from breadcrumbs 'df_topics_breadcrumbs'...

* Calculating topics_features dataframe...


Creating and cleaning topic features...

Created 'topic_features'


,channel,category,level,language,has_content,content_ids,sentences
t_00004da3a1b2,000cf7,source,four,bg,True,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...,Откриването на резисторите.Изследване на матер...
t_00068291e9a4,8e286a,source,four,pt,True,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...,Entradas e saídas de uma função.Entenda um pou...
t_00069b63a70a,6e3ba4,source,three,en,True,c_11a1dc0bfb99,Transcripts.
t_0006d41a73a8,000cf7,source,four,bg,True,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...,Графики на експоненциални функции Алгебра 2 н...



* Calculating content_features dataframe...


Creating and cleaning content features...

Created 'content_features'


,kind,language,sentences
id,,,
c_00002381196d,video,es,Sumar números de varios dígitos 48 029 233 93...
c_000087304a9e,video,it,Trovare i fattori di un numero.Sal trova i fat...
c_0000ad142ddb,video,es,Sumar curvas de demanda.Cómo añadir curvas de ...
c_0000c03adc8d,document,pt,Nado de aproximação.Neste vídeo você vai apren...
c_00016694ea2a,document,es,geometry m3 topic a overview pdf.geometry m3 t...


In [60]:
#Calculating embeddings from text
print ('\n* Calculating embeddings for both topics and contents text columns...\n')
topics_embeds, contents_embeds, topic_id_embeds, content_ids_embeds = get_embeddings(topics_features, contents_features)


* Calculating embeddings for both topics and contents text columns...


Getting embeddings for 4 sentences from topics

Getting embeddings for 1000 sentences from contents


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

# Getting topics-contents from NN  

In [86]:
#Calculating unsupervised k neighbours contents, for topics
print ('\n* Calculating k neighnors predictions...\n')
model, preds, ground_truth, fscore = k_neighbors (K, topics_embeds, contents_embeds, topic_id_embeds, content_ids_embeds, df_corr)
preds.to_csv('knn_predictions.csv', index=False)
print ('\n All finished!\n')
print ('\n*** Predictions ***\n')
display(preds)
print ('\n*** Score for KNN only ***\n')
display(fscore)


* Calculating k neighnors predictions...


 K value: 100

Fitting KNN model...

Getting content_ids for 4 topics... 

Calculating scores...

Topics to match content: 5

Correct predictions: 0

F2 Score: 0.0



 All finished!


*** Predictions ***



,topic_id,content_ids
0,t_00004da3a1b2,c_005a7c3eadf3 c_004312ef7ecd c_0041092041cd c...
1,t_00068291e9a4,c_0173e0b292fb c_00b2e5bc0065 c_019c6cf2cdac c...
2,t_00069b63a70a,c_000f7fcdaacc c_0112c0ea6b55 c_00c05add5064 c...
3,t_0006d41a73a8,c_0124fb4b510b c_01b0c3e81481 c_010c8b0e946f c...



*** Score for KNN only ***



,topic_id,content_ids_pred,content_ids_actual,correct_pred,precision,recall,f0.5,f1,f2
0,t_00004da3a1b2,"[c_005a7c3eadf3, c_004312ef7ecd, c_0041092041c...","[c_1108dd0c7a5d, c_376c5a8eb028, c_5bc0e1e2cba...",0,0.0,0.0,0.0,0.0,0.0
1,t_00068291e9a4,"[c_0173e0b292fb, c_00b2e5bc0065, c_019c6cf2cda...","[c_639ea2ef9c95, c_89ce9367be10, c_ac1672cdcd2...",0,0.0,0.0,0.0,0.0,0.0
2,t_00069b63a70a,"[c_000f7fcdaacc, c_0112c0ea6b55, c_00c05add506...",[c_11a1dc0bfb99],0,0.0,0.0,0.0,0.0,0.0
3,t_0006d41a73a8,"[c_0124fb4b510b, c_01b0c3e81481, c_010c8b0e946...","[c_0c6473c3480d, c_1c57a1316568, c_5e375cf14c4...",0,0.0,0.0,0.0,0.0,0.0


# Setup Features and Target

In [125]:
# Get features and target, split into train and validation sets
preds_expl = get_target(preds)
X, y_target = get_features (preds_expl, topics_features, contents_features)
X_train, X_val, y_train, y_val = train_test_split (X, y_target, random_state = 42)
print ('\nDisplaying datasets:\n\n')
display (X_train.head())
display (y_train.head())
display (X_val.head())
display (y_val.head())
print (f'Shapes: X_train {X_train.shape}, y_train {y_train.shape}, X_val {X_val.shape}, y_val {y_val.shape}')


Sorting features for X...


Sorting binary y_target...



X and y_target are defined.

Displaying datasets:




,topic_id,content_ids,match,channel,category,level,language,has_content,topic_title,kind,sentences
286,t_00069b63a70a,c_018659c04857,0,6e3ba4,source,three,en,True,Transcripts,video,Subsidairy Book Analytical Petty Cash Book I.
65,t_00004da3a1b2,c_012ec315a05c,0,000cf7,source,four,bg,True,Откриването на резисторите,video,يتعر ف مفهوم العلاقة والإقتران الاقتران أو ال...
341,t_0006d41a73a8,c_00d3ad407855,0,000cf7,source,four,bg,True,Графики на експоненциални функции Алгебра 2 н...,document,Precalculus Module 1 Topic A Lesson 5 Teach...
392,t_0006d41a73a8,c_01997eee2b70,0,000cf7,source,four,bg,True,Графики на експоненциални функции Алгебра 2 н...,video,Хипоталамус и хипофизна жлеза.Какво кара ендок...
21,t_00004da3a1b2,c_006694575048,0,000cf7,source,four,bg,True,Откриването на резисторите,video,Axe de symétrie.Qu est ce qu un axe de symétri...


286    0
65     0
341    0
392    0
21     0
Name: match, dtype: int64

,topic_id,content_ids,match,channel,category,level,language,has_content,topic_title,kind,sentences
262,t_00069b63a70a,c_012531c609f8,0,6e3ba4,source,three,en,True,Transcripts,html5,Induction Proof Greeting with Handshakes.
242,t_00069b63a70a,c_00e0a8939f54,0,6e3ba4,source,three,en,True,Transcripts,document,Grade 10 ELA Module 4 Unit 1 Overview.Grade 1...
308,t_0006d41a73a8,c_002624bb326b,0,000cf7,source,four,bg,True,Графики на експоненциални функции Алгебра 2 н...,document,math g7 m6 topic e lesson 27 student spanish p...
256,t_00069b63a70a,c_011616a5716c,0,6e3ba4,source,three,en,True,Transcripts,video,Study of Detergents.
368,t_0006d41a73a8,c_014d183f009c,0,000cf7,source,four,bg,True,Графики на експоненциални функции Алгебра 2 н...,html5,I Am Latvia Research Report ..Jump to navigat...


262    0
242    0
308    0
256    0
368    0
Name: match, dtype: int64

Shapes: X_train (300, 11), y_train (300,), X_val (100, 11), y_val (100,)


In [126]:
# save out features and target
DATA_PATH = Path.cwd().parent.resolve()
work_path = DATA_PATH / 'working'

x_train_pth = work_path / "x_train.csv"
x_train_pth.parent.mkdir(exist_ok=True, parents=True)
y_train_pth = work_path / "y_train.csv"
x_val_pth = work_path / "x_val.csv"
y_val_pth = work_path / "y_val.csv"

X_train.to_csv (x_train_pth, index=False)
X_val.to_csv (x_val_pth, index=False)
y_train.to_csv (y_train_pth, index=False)
y_val.to_csv (y_val_pth, index=False)
print (f"\nTraining and validation sets saved in files at {work_path}.\n\n")


Training and validation sets saved in files at /kaggle/working.




# Using LGBMClassifier

As a classifier, the choice is LightGBM's LGBMClassifier.

It has the advantage of not needing hot-encoding and being 8-times faster than the same OneHotEncoder from sklearn.

Below, the %%writefile line magic is used to write out a training script. Then ! to run a shell command from the jupyter notebook, and kick off the script. This is due to a [bug](https://github.com/microsoft/LightGBM/issues/4229).

We'll use the typer package to make our script easy to run in the command line.

In [115]:
%%writefile train_gbm_model.py
import lightgbm as lgb

import joblib
import pandas as pd
import numpy as np
from pathlib import Path
from loguru import logger
import typer

DATA_PATH = Path.cwd().parent.resolve()
MODEL_DIR = DATA_PATH / 'working'

def main(
    features_path= MODEL_DIR / "x_train.csv",
    labels_path=MODEL_DIR / "y_train.csv",
    model_save_path=MODEL_DIR / "lgb_classifier.txt",
):
    """
    Train a LightGBM model based on training features in features_path and
    training labels in labels_path. Save our the trained model to model_save_path
    """

    # load saved features and labels
    X_train = pd.read_csv (features_path, dtype="category")
    y_train = pd.read_csv (labels_path, dtype=int)
    
    logger.info(f"Loaded training features of shape {X_train.shape} from {features_path}")
    logger.info(f"Loading training labels of shape {y_train.shape} from {labels_path}")
    # instantiate tree model
    model = lgb.LGBMClassifier(
        boosting_type='gbdt',
        objective='binary',
        num_leaves= 50,
        random_state=10)

    # fit model
    fit_params={
            'feature_name': 'auto',
            'categorical_feature': 'auto'
           }
    
    logger.info("Fitting LGBM model")
    model.fit(X_train, y_train, **fit_params)
    print(model)

    # save out model weights
    joblib.dump(model, str(model_save_path))
    logger.success(f"Model weights saved to {model_save_path}")


if __name__ == "__main__":
    typer.run(main)

Overwriting train_gbm_model.py


In [92]:
%%writefile train_gbm_model.py
import lightgbm as lgb

import joblib
import pandas as pd
import numpy as np
from pathlib import Path
from loguru import logger
import typer

DATA_PATH = Path.cwd().parent.resolve()
MODEL_DIR = DATA_PATH / 'working'

def main(
    features_path= MODEL_DIR / "x_train.csv",
    labels_path=MODEL_DIR / "y_train.csv",
    model_save_path=MODEL_DIR / "lgb_classifier.txt",
):
    """
    Train a LightGBM model based on training features in features_path and
    training labels in labels_path. Save our the trained model to model_save_path
    """

    # load saved features and labels
    X_train = pd.read_csv (features_path, dtype="category")
    y_train = pd.read_csv (labels_path, dtype=int)

    logger.info(f"Loaded training features of shape {X_train.shape} from {features_path}")
    logger.info(f"Loading training labels of shape {y_train.shape} from {labels_path}")
    
    hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'recall',
    'learning_rate': 0.1,
    "num_leaves": 20,  
    "max_bin": 100,
    }

    lgb_train = lgb.Dataset(X_train, y_train)    
    logger.info("Fitting LGBM model")
    

    # instantiate tree model and train
    model = lgb.train(hyper_params, lgb_train, num_boost_round=10)

    # save out model weights
    joblib.dump(model, str(model_save_path))
    logger.success(f"Model weights saved to {model_save_path}")


if __name__ == "__main__":
    typer.run(main)

Overwriting train_gbm_model.py


In [116]:
!python train_gbm_model.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2023-01-30 23:53:26.345 | INFO     | __main__:main:27 - Loaded training features of shape (300, 11) from /kaggle/working/x_train.csv
2023-01-30 23:53:26.345 | INFO     | __main__:main:28 - Loading training labels of shape (300, 1) from /kaggle/working/y_train.csv
2023-01-30 23:53:26.345 | INFO     | __main__:main:42 - Fitting LGBM model
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:13

# Validation predictions

In [117]:
%%writefile predict_gbm_model.py
import lightgbm as lgb

import joblib
from loguru import logger
import numpy as np
import pandas as pd
from pathlib import Path
import typer

DATA_PATH = Path.cwd().parent.resolve()
MODEL_DIR = DATA_PATH / 'working'

def main(
    model_weights_path=MODEL_DIR / "lgb_classifier.txt",
    features_path=MODEL_DIR / "x_val.csv",
    preds_save_path=MODEL_DIR / "y_preds.npy",
):
    """
    Generate predictions with a LightGBM model using weights saved at model_weights_path
    and features saved at features_path. Save out predictions to preds_save_path.
    """
    # load model weights
    lgb_model = joblib.load(model_weights_path)
    logger.info(f"Loaded model {lgb_model} from {model_weights_path}")

    # load the features
    X_val = pd.read_csv (features_path, dtype="category")
    logger.info(f"Loaded features of shape {X_val.shape} from {features_path}")

    # generate predictions
    y_preds = lgb_model.predict(X_val)

    # save out predictions
    with open(preds_save_path, "wb") as f:
        np.save(f, y_preds)
    logger.success(f"Predictions saved to {preds_save_path}")


if __name__ == "__main__":
    typer.run(main)

Overwriting predict_gbm_model.py


In [118]:
!python predict_gbm_model.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2023-01-30 23:53:35.640 | INFO     | __main__:main:24 - Loaded model LGBMClassifier(num_leaves=50, objective='binary', random_state=10) from /kaggle/working/lgb_classifier.txt
2023-01-30 23:53:35.674 | INFO     | __main__:main:28 - Loaded features of shape (100, 11) from /kaggle/working/x_val.csv
2023-01-30 23:53:35.694 | SUCCESS  | __main__:main:36 - Predictions saved to /kaggle/working/y_preds.npy


# Model Scoring

In [197]:
preds_pth = work_path / "y_preds.npy"
with open(preds_pth, "rb") as f:
    y_preds = np.load(f)

In [212]:
df = X_val[y_preds.astype(bool)].iloc[:, [0,1]]
y_final_preds=df.set_index('topic_id').groupby(level=0).agg(lambda x: ' '.join(x)).reset_index()
display (y_final_preds)
df = X_val[y_val.astype(bool)].iloc[:, [0,1]]
y_final_val=df.set_index('topic_id').groupby(level=0).agg(lambda x: ' '.join(x)).reset_index()
display (y_final_val)

,topic_id,content_ids
0,t_00004da3a1b2,c_0143c0f8a522 c_00d2cf51f5f0 c_000ba728719e
1,t_00068291e9a4,c_0058a31b8ecf
2,t_00069b63a70a,c_009fa1daa174 c_014990ad8fbc
3,t_0006d41a73a8,c_016cb9cf0b7b


,topic_id,content_ids


In [211]:
y_preds = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [213]:
y_final_val = y_final_preds

In [216]:
# Calculate the scoring for the final classification model
display(y_final_val)
display(y_final_preds)
calculate_Fscore(y_final_preds, y_final_val)

,topic_id,content_ids_pred
0,t_00004da3a1b2,"[c_0143c0f8a522, c_00d2cf51f5f0, c_000ba728719e]"
1,t_00068291e9a4,[c_0058a31b8ecf]
2,t_00069b63a70a,"[c_009fa1daa174, c_014990ad8fbc]"
3,t_0006d41a73a8,[c_016cb9cf0b7b]


,topic_id,content_ids_pred
0,t_00004da3a1b2,"[c_0143c0f8a522, c_00d2cf51f5f0, c_000ba728719e]"
1,t_00068291e9a4,[c_0058a31b8ecf]
2,t_00069b63a70a,"[c_009fa1daa174, c_014990ad8fbc]"
3,t_0006d41a73a8,[c_016cb9cf0b7b]


AttributeError: 'DataFrame' object has no attribute 'content_ids'

# Submissions

In [ ]:
#import pickle
#pickle.dump(model, open(f'ST-KNN-{k[1]}', 'wb'))
preds.to_csv('submission.csv', index=False)